In [27]:
%matplotlib inline
import pandas as pd
import numpy as np
from IPython.display import display
from scipy.stats import chi2_contingency
from os.path import exists
from missforest.missforest import MissForest

# ensure reproducibility
np.random.seed(123)


In [28]:
pkl_file = './df.pkl'
df = pd.read_pickle(pkl_file)
df.shape

(608, 255)

In [29]:
col_sleep = [
  'BL_ODI_sleeping',
  '3m_ODI_sleeping',
  '12m_ODI_sleeping',
  '24m_ODI_sleeping',
  '36m_ODI_sleeping'
]

df.loc[:, col_sleep].describe()

,BL_ODI_sleeping,3m_ODI_sleeping,12m_ODI_sleeping,24m_ODI_sleeping,36m_ODI_sleeping
count,602.000000,537.000000,471.000000,460.000000,157.000000
mean,1.965116,0.945996,0.868365,-42.593478,1.000000
std,1.309296,1.147218,1.133474,658.655091,1.182132
min,0.000000,0.000000,0.000000,-9999.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,1.000000,0.000000,1.000000,1.000000
75%,3.000000,1.000000,1.000000,1.000000,1.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000


In [30]:
df['>24m_ODI_sleeping'] = df['24m_ODI_sleeping'].combine_first(df['36m_ODI_sleeping'])

In [31]:
df.loc[:, col_sleep + ['>24m_ODI_sleeping']].describe()

,BL_ODI_sleeping,3m_ODI_sleeping,12m_ODI_sleeping,24m_ODI_sleeping,36m_ODI_sleeping,>24m_ODI_sleeping
count,602.000000,537.000000,471.000000,460.000000,157.000000,531.000000
mean,1.965116,0.945996,0.868365,-42.593478,1.000000,-36.725047
std,1.309296,1.147218,1.133474,658.655091,1.182132,613.134471
min,0.000000,0.000000,0.000000,-9999.000000,0.000000,-9999.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,1.000000,0.000000,1.000000,1.000000,1.000000
75%,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [32]:
col_sleep = [
  'BL_ODI_sleeping',
  '3m_ODI_sleeping',
  '12m_ODI_sleeping',
  '>24m_ODI_sleeping'
]
df = df.dropna(subset=col_sleep[-1])
df = df.drop(df[df[col_sleep[-1]] < 0].index)
df.shape

(529, 256)

In [33]:
pkl_file = './df_sleep_mv.pkl'

na_cols = ['3m_ODI_sleeping','12m_ODI_sleeping']

if exists(pkl_file):
  df_mf = pd.read_pickle(pkl_file)
else:
  mf = MissForest()
  df_na_scores_filled = mf.fit_transform(df[na_cols].copy())
  df_mf = df.copy()
  df_mf[na_cols] = df_na_scores_filled
  df_mf[col_sleep] = df_mf[col_sleep].round().astype('Int64')
  df_mf.to_pickle(pkl_file)

print('Any null values:', df_mf[col_sleep].isnull().values.any())
display(df_mf[col_sleep].head())

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000299 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6
[LightGBM] [Info] Number of data points in the train set: 405, number of used features: 1
[LightGBM] [Info] Start training from score 0.871605
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

,BL_ODI_sleeping,3m_ODI_sleeping,12m_ODI_sleeping,>24m_ODI_sleeping
0,1,0,0,0
1,0,0,0,0
2,2,0,1,0
9,1,1,1,1
10,1,1,0,0


In [34]:
def value_to_pct(col):
  tmp = col.value_counts()
  for i in range(6):
    try:
      tmp.loc[i]
    except KeyError:
      tmp.loc[i] = 0.0
  values = tmp.astype(str)
  pct = col.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
  return (values + ' (' + pct + ')')

In [35]:
df_mf.columns.to_list()

['id',
 'site',
 'site_id',
 'date_sx',
 'date_sx_A',
 'age',
 'sex',
 'principal_spondy_dx',
 'grade_listhesis',
 'height',
 'weight',
 'insurance',
 'prev_sx',
 'smoker',
 'diabetes',
 'cad',
 'anxiety',
 'depression',
 'osteoporosis',
 'main_symptom',
 'motor_deficit',
 'ambulation',
 'symptom_duration',
 'bmi',
 'ethniticity',
 'education',
 'workers_comp',
 'employment',
 'unemployed_status',
 'asa_grade',
 'surg_approach',
 'laminectomy_performed',
 'laminectomy_levels',
 'arthrodesis_performed',
 'arthrodesis_levels',
 'interbody_graft',
 'MIS_decompression',
 'MIS_percutaneous_pedicle_screws',
 'MIS_pedicle_screws',
 'cortical_screws',
 'MIS_interbody',
 'blood_loss',
 'length_of_surgery',
 'los',
 'place_discharged_to',
 'place_discharged_to_facility',
 '3m_pt_satisfaction',
 '12m_pt_satisfaction',
 '24m_pt_satisfaction',
 '36m_pt_satisfaction',
 'latest_pt_satisfaction',
 '3m_return_to_work',
 '12m_return_to_work',
 '24m_return_to_work',
 '36m_return_to_work',
 'latest_return

In [37]:
bool_impaired_BL = df_mf['BL_ODI_sleeping'] > 0
bool_normal_BL = df_mf['BL_ODI_sleeping'] == 0

In [38]:
df_mf.loc[bool_impaired_BL].shape

(455, 256)

In [43]:
df_mf['>24m_ODI_sleeping_improved'] = df_mf[bool_impaired_BL]['BL_ODI_sleeping'] > df_mf.loc[bool_impaired_BL]['>24m_ODI_sleeping']

In [44]:
df_mf['>24m_ODI_sleeping_improved'].value_counts()

>24m_ODI_sleeping_improved
True     319
False    136
Name: count, dtype: Int64

# Univariate comparison

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")


In [116]:
def floatToSF(value:float, sf=3):
  rounded_value = round(value, sf)
  return f"{rounded_value}**" if rounded_value < 0.05 else rounded_value

def univariateAnalysis(cols: list[str], tests: list):
  data = {
    'improved function': [],
    'same or worse function': [],
    'P value': []}
  for col, test in zip(cols, tests):
    series1 = df_mf.loc[df_mf['>24m_ODI_sleeping_improved']][col].dropna()
    series2 = df_mf.loc[~df_mf['>24m_ODI_sleeping_improved']][col].dropna()
    _, p_value = test(series1, series2)
    data['improved function'].append(f'{round(series1.mean(), 1)} ± {round(np.std(series1), 1)}')
    data['same or worse function'].append(f'{round(series2.mean(), 1)} ± {round(np.std(series2), 1)}')
    data['P value'].append(floatToSF(p_value))


  display(pd.DataFrame(data, index=cols))

In [107]:
df_mf['sex'].value_counts()

sex
2    310
1    219
Name: count, dtype: int64

In [111]:
df_mf['sex'] = df_mf['sex'].replace({2: 'F', 1: 'M'})

In [114]:
from scipy.stats import chi2_contingency
# Create a contingency table using the crosstab function
contingency_table = pd.crosstab(df_mf['sex'], df_mf['>24m_ODI_sleeping_improved'])

# Display the contingency table
print(contingency_table)
chi2_stat, p_value, _, _ = chi2_contingency(contingency_table)
print(f'Chi-Square Statistic: {chi2_stat}, p-value: {floatToSF(p_value)}')

>24m_ODI_sleeping_improved  False  True 
sex                                     
F                              76    195
M                              60    124
Chi-Square Statistic: 0.8826054969045581, p-value: 0.347


In [136]:
from scipy.stats import ttest_ind, mannwhitneyu, wilcoxon

univariateAnalysis(['age', 
                    'bmi',
                    'BL_ODI',
                    'blood_loss', 
                    'length_of_surgery', 
                    'los'], 
                   [ttest_ind] * 6)

,improved function,same or worse function,P value
age,62.2 ± 11.9,60.9 ± 11.9,0.291
bmi,30.1 ± 6.3,30.9 ± 6.2,0.21
BL_ODI,49.3 ± 15.7,45.4 ± 16.4,0.019**
blood_loss,183.3 ± 186.4,231.4 ± 247.2,0.027**
length_of_surgery,177.7 ± 84.2,185.6 ± 89.6,0.382
los,2.9 ± 1.8,2.8 ± 1.7,0.431


In [122]:
df_mf.loc[:, ['reop_30day', 'reop_1year', 'reop_2year']].describe()

,reop_30day,reop_1year,reop_2year
count,481.000000,529.000000,529.000000
mean,0.031185,0.041588,0.047259
std,0.173998,0.199835,0.212393
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000


# Multivariable analysis

In [201]:
multivar_cols = [
  'age',
  'bmi',
  'main_symptom',
  # 'motor_deficit',
  'asa_grade',
  'BL_ODI',
  'MIS_decompression',
  'MIS_percutaneous_pedicle_screws',
  'MIS_pedicle_screws',
  'MIS_interbody',
  'education',
  'length_of_surgery',
  'reop_2year',
  '>24m_ODI_sleeping_improved'
]

df_mv = df_mf.loc[:, multivar_cols].copy()
df_mv = df_mv.dropna(subset=['>24m_ODI_sleeping_improved'])
df_mv['length_of_surgery'] = df_mv['length_of_surgery'].fillna(df_mv['length_of_surgery'].mean())

main_symptom_map = {
  1: 'back dominant',
  2: 'leg dominant',
  3: 'back-leg'
}
df_mv['leg_dominant'] = (df_mv['main_symptom'] == 2).astype(int)
# df_mv['motor_deficit'] = df_mv['motor_deficit'].astype(int)
df_mv['asa_grade_12_vs_34'] = df_mv['asa_grade'].apply(lambda x: 0 if x < 3 else 1)

df_multivar_mis_bool = df_mv.loc[:, [
  'MIS_decompression',
  'MIS_percutaneous_pedicle_screws',
  'MIS_pedicle_screws',
  'MIS_interbody'
]].any(axis=1)
df_mv['MIS'] = df_multivar_mis_bool.astype(int)
df_mv['>4yrs_education'] = ((df_mv['education'] == 4.0) | (df_mv['education'] == 5.0)).astype(int)
df_mv['>24m_ODI_sleeping_improved'] = df_mv['>24m_ODI_sleeping_improved'].astype(int)

multivar_cols2 = [
  'age',
  'bmi',
  'leg_dominant',
  # 'motor_deficit',
  'asa_grade_12_vs_34',
  'BL_ODI',
  'MIS',
  '>4yrs_education',
  'length_of_surgery',
  'reop_2year',
  '>24m_ODI_sleeping_improved'
]
display(df_mv.loc[:, multivar_cols2].head(3))
display(df_mv.loc[:, multivar_cols2].describe())
print(df_mv.shape)

,age,bmi,leg_dominant,asa_grade_12_vs_34,BL_ODI,MIS,>4yrs_education,length_of_surgery,reop_2year,>24m_ODI_sleeping_improved
0,68.35,30.52,0,0,20.000000,1,0,69.0,0,1
2,71.28,28.99,1,0,44.444444,0,0,182.0,0,1
9,84.34,25.20,0,1,37.500000,1,0,71.0,0,0


,age,bmi,leg_dominant,asa_grade_12_vs_34,BL_ODI,MIS,>4yrs_education,length_of_surgery,reop_2year,>24m_ODI_sleeping_improved
count,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000
mean,61.824746,30.296541,0.204396,0.415385,48.141778,0.413187,0.362637,180.013793,0.048352,0.701099
std,11.892246,6.299201,0.403703,0.493331,16.041222,0.492948,0.481291,84.083501,0.214744,0.458280
min,20.800000,16.640000,0.000000,0.000000,10.000000,0.000000,0.000000,26.000000,0.000000,0.000000
25%,54.310000,25.552015,0.000000,0.000000,38.000000,0.000000,0.000000,123.500000,0.000000,0.000000
50%,62.220000,29.480000,0.000000,0.000000,48.000000,0.000000,0.000000,175.000000,0.000000,1.000000
75%,70.200000,33.902609,0.000000,1.000000,60.000000,1.000000,1.000000,222.000000,0.000000,1.000000
max,95.000000,56.333313,1.000000,1.000000,90.000000,1.000000,1.000000,812.000000,1.000000,1.000000


(455, 17)


In [202]:
df_mv.loc[:, multivar_cols2].isna().describe()

,age,bmi,leg_dominant,asa_grade_12_vs_34,BL_ODI,MIS,>4yrs_education,length_of_surgery,reop_2year,>24m_ODI_sleeping_improved
count,455,455,455,455,455,455,455,455,455,455
unique,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False
freq,455,455,455,455,455,455,455,455,455,455


In [229]:
import statsmodels.api as sm

y = df_mv['>24m_ODI_sleeping_improved']
X = df_mv[[
      'asa_grade_12_vs_34',
      'age',
      'bmi',
      '>4yrs_education',
      'leg_dominant',
      'length_of_surgery',
      'MIS',
      'BL_ODI',
      'reop_2year']]

res = sm.GLM(y, sm.add_constant(X)).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                      
======================================================================================
Dep. Variable:     >24m_ODI_sleeping_improved   No. Observations:                  455
Model:                                    GLM   Df Residuals:                      445
Model Family:                        Gaussian   Df Model:                            9
Link Function:                       Identity   Scale:                         0.20314
Method:                                  IRLS   Log-Likelihood:                -277.95
Date:                        Tue, 09 Jan 2024   Deviance:                       90.396
Time:                                16:57:36   Pearson chi2:                     90.4
No. Iterations:                             3   Pseudo R-squ. (CS):            0.05242
Covariance Type:                    nonrobust                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.4224      0.196      2.153      0.031       0.038       0.807
asa_grade_12_vs_34    -0.0807      0.047     -1.715      0.086      -0.173       0.012
age                    0.0021      0.002      1.054      0.292      -0.002       0.006
bmi                   -0.0020      0.004     -0.528      0.598      -0.009       0.005
>4yrs_education        0.0371      0.045      0.824      0.410      -0.051       0.125
leg_dominant           0.0678      0.055      1.222      0.222      -0.041       0.176
length_of_surgery     -0.0001      0.000     -0.462      0.644      -0.001       0.000
MIS                    0.0709      0.044      1.622      0.105      -0.015       0.157
BL_ODI                 0.0045      0.001      3.180      0.001       0.002       0.007
reop_2year            -0.1588      0.101     -1.577      0.115      -0.356       0.039
======================================================================================
"""

In [236]:
def print_OR(results):
  conf = results.conf_int()
  conf['OR'] = results.params
  conf.columns = ['5%', '95%', 'OR']
  df_pp = np.exp(conf[['OR', '5%', '95%']])
  df_pp = df_pp.drop(df_pp.index[0])
  df_pp['p'] = results.pvalues
  df_pp = df_pp.round(2)
  df_pp['p'] = df_pp['p'].apply(lambda x: f'{x}**' if x < 0.05 else x)
  df_pp['Adjusted OR (95% CI)'] = df_pp.apply(lambda x: f"{x['OR']} ({x['5%']}-{x['95%']})", axis=1)
  return df_pp[['Adjusted OR (95% CI)', 'p']]

In [231]:
print_OR(res)

,Adjusted OR (95% CI),p
asa_grade_12_vs_34,0.922 (0.841-1.012),0.086
age,1.002 (0.998-1.006),0.292
bmi,0.998 (0.991-1.005),0.598
>4yrs_education,1.038 (0.95-1.133),0.41
leg_dominant,1.07 (0.96-1.193),0.222
length_of_surgery,1.0 (0.999-1.0),0.644
MIS,1.073 (0.985-1.169),0.105
BL_ODI,1.005 (1.002-1.007),0.001**
reop_2year,0.853 (0.7-1.039),0.115


# Subgroup Multivariable Analysis

In [232]:
import statsmodels.api as sm

y = df_mv.loc[df_mv['age'] < 65]['>24m_ODI_sleeping_improved']
X = df_mv.loc[df_mv['age'] < 65][[
      'asa_grade_12_vs_34',
      # 'age',
      'bmi',
      '>4yrs_education',
      'leg_dominant',
      'length_of_surgery',
      'MIS',
      'BL_ODI',
      'reop_2year']]

res_lt_65 = sm.GLM(y, sm.add_constant(X)).fit()
res_lt_65.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                      
======================================================================================
Dep. Variable:     >24m_ODI_sleeping_improved   No. Observations:                  269
Model:                                    GLM   Df Residuals:                      260
Model Family:                        Gaussian   Df Model:                            8
Link Function:                       Identity   Scale:                         0.21112
Method:                                  IRLS   Log-Likelihood:                -167.92
Date:                        Tue, 09 Jan 2024   Deviance:                       54.890
Time:                                16:57:59   Pearson chi2:                     54.9
No. Iterations:                             3   Pseudo R-squ. (CS):            0.07381
Covariance Type:                    nonrobust                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.4756      0.170      2.798      0.005       0.142       0.809
asa_grade_12_vs_34    -0.1378      0.062     -2.227      0.026      -0.259      -0.016
bmi                   -0.0024      0.004     -0.543      0.587      -0.011       0.006
>4yrs_education        0.1159      0.062      1.876      0.061      -0.005       0.237
leg_dominant           0.1366      0.085      1.605      0.109      -0.030       0.304
length_of_surgery  -8.258e-05      0.000     -0.261      0.794      -0.001       0.001
MIS                    0.0316      0.059      0.540      0.589      -0.083       0.146
BL_ODI                 0.0055      0.002      3.127      0.002       0.002       0.009
reop_2year            -0.0735      0.118     -0.620      0.535      -0.306       0.159
======================================================================================
"""

In [237]:
print_OR(res_lt_65)

,Adjusted OR (95% CI),p
asa_grade_12_vs_34,0.87 (0.77-0.98),0.03**
bmi,1.0 (0.99-1.01),0.59
>4yrs_education,1.12 (0.99-1.27),0.06
leg_dominant,1.15 (0.97-1.35),0.11
length_of_surgery,1.0 (1.0-1.0),0.79
MIS,1.03 (0.92-1.16),0.59
BL_ODI,1.01 (1.0-1.01),0.0**
reop_2year,0.93 (0.74-1.17),0.54


In [234]:
import statsmodels.api as sm

y = df_mv.loc[df_mv['age'] >= 65]['>24m_ODI_sleeping_improved']
X = df_mv.loc[df_mv['age'] >= 65][[
      'asa_grade_12_vs_34',
      # 'age',
      'bmi',
      '>4yrs_education',
      'leg_dominant',
      'length_of_surgery',
      'MIS',
      'BL_ODI',
      'reop_2year']]

res_gte_65 = sm.GLM(y, sm.add_constant(X)).fit()
res_gte_65.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                      
======================================================================================
Dep. Variable:     >24m_ODI_sleeping_improved   No. Observations:                  186
Model:                                    GLM   Df Residuals:                      177
Model Family:                        Gaussian   Df Model:                            8
Link Function:                       Identity   Scale:                         0.18862
Method:                                  IRLS   Log-Likelihood:                -104.18
Date:                        Tue, 09 Jan 2024   Deviance:                       33.386
Time:                                16:57:59   Pearson chi2:                     33.4
No. Iterations:                             3   Pseudo R-squ. (CS):            0.06265
Covariance Type:                    nonrobust                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.4547      0.213      2.131      0.033       0.036       0.873
asa_grade_12_vs_34    -0.0074      0.070     -0.107      0.915      -0.144       0.129
bmi                    0.0027      0.007      0.377      0.706      -0.011       0.017
>4yrs_education       -0.0498      0.067     -0.743      0.457      -0.181       0.081
leg_dominant           0.0267      0.072      0.370      0.712      -0.115       0.168
length_of_surgery  -1.756e-05      0.000     -0.038      0.969      -0.001       0.001
MIS                    0.1344      0.065      2.053      0.040       0.006       0.263
BL_ODI                 0.0040      0.002      1.636      0.102      -0.001       0.009
reop_2year            -0.3140      0.202     -1.552      0.121      -0.711       0.083
======================================================================================
"""

In [238]:
print_OR(res_gte_65)

,Adjusted OR (95% CI),p
asa_grade_12_vs_34,0.99 (0.87-1.14),0.91
bmi,1.0 (0.99-1.02),0.71
>4yrs_education,0.95 (0.83-1.08),0.46
leg_dominant,1.03 (0.89-1.18),0.71
length_of_surgery,1.0 (1.0-1.0),0.97
MIS,1.14 (1.01-1.3),0.04**
BL_ODI,1.0 (1.0-1.01),0.1
reop_2year,0.73 (0.49-1.09),0.12
